In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
from PIL import Image
import random
import numpy as np
from tqdm import tqdm

SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
# Dataset parameters and class abbreviation mapping
DATASET_DIR = r"Dataset/BreaKHis/breast"
MAGNIFICATION = "200X"
BATCH_SIZE = 24
NUM_WORKERS = 0
NUM_EPOCHS = 150
USE_AMP = True
VALID_SPLIT = 0.15  

CLASS_ABBR_MAP = {
    "adenosis": "A", "fibroadenoma": "F", "phyllodes_tumor": "PT", "tubular_adenoma": "TA",
    "ductal_carcinoma": "DC", "lobular_carcinoma": "LC", "mucinous_carcinoma": "MC", "papillary_carcinoma": "PC"
}

RESIZE_SIZE = (128, 128)

In [3]:
class BreakHisDataset(Dataset):
    def __init__(self, root_dir, magnification, transform=None):
        self.samples = []
        self.transform = transform
        for binary_class in ["benign", "malignant"]:
            sob_path = os.path.join(root_dir, binary_class, "SOB")
            if not os.path.exists(sob_path): continue
            for class_folder in os.listdir(sob_path):
                class_abbr = CLASS_ABBR_MAP.get(class_folder.lower(), class_folder)
                class_path = os.path.join(sob_path, class_folder)
                if not os.path.isdir(class_path): continue
                for patient_folder in os.listdir(class_path):
                    patient_path = os.path.join(class_path, patient_folder)
                    if not os.path.isdir(patient_path): continue
                    mag_path = os.path.join(patient_path, magnification)
                    if os.path.exists(mag_path):
                        images = [f for f in os.listdir(mag_path)
                                  if f.lower().endswith((".png", ".jpg", ".jpeg"))]
                        for img_name in images:
                            img_path = os.path.join(mag_path, img_name)
                            label = 0 if binary_class == "benign" else 1
                            # keep original path + label in samples
                            self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        image = image.resize(RESIZE_SIZE, Image.LANCZOS)
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
# Transforms
train_transform = transforms.Compose([
    transforms.RandomRotation(40),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomAffine(degrees=0, shear=0.2),
    transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

def split_indices(n, valid_split=0.15, test_split=0.15):
    indices = np.arange(n)
    np.random.shuffle(indices)
    test_size = int(n * test_split)
    valid_size = int(n * valid_split)
    train_size = n - test_size - valid_size
    train_idx = indices[:train_size]
    valid_idx = indices[train_size:train_size+valid_size]
    test_idx = indices[train_size+valid_size:]
    return train_idx, valid_idx, test_idx

full_dataset = BreakHisDataset(DATASET_DIR, MAGNIFICATION, transform=None)
train_idx, valid_idx, test_idx = split_indices(len(full_dataset), valid_split=VALID_SPLIT, test_split=0.15)

train_dataset = Subset(full_dataset, train_idx)
valid_dataset = Subset(full_dataset, valid_idx)
test_dataset = Subset(full_dataset, test_idx)

class WrappedDataset(Dataset):
    def __init__(self, subset, transform):
        self.subset = subset
        self.transform = transform
    def __len__(self):
        return len(self.subset)
    def __getitem__(self, idx):
        x, y = self.subset[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

train_dataset = WrappedDataset(train_dataset, train_transform)
valid_dataset = WrappedDataset(valid_dataset, test_transform)
test_dataset = WrappedDataset(test_dataset, test_transform)

In [5]:
# Weighted sampler for training
labels = [y for _, y in train_dataset]
class_counts = np.bincount(labels)
class_weights = 1. / class_counts
weights = [class_weights[label] for label in labels]
sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=NUM_WORKERS, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

In [6]:
# Model blocks (InceptionRCLBlock, TransitionUnit, IRRCNN) - same as شما
class InceptionRCLBlock(nn.Module):
    def __init__(self, in_channels, out_channels, t_steps=2, activation="relu"):
        super().__init__()
        self.t_steps = t_steps
        self.activation = activation
        self.branch1x1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0)
        self.rcl_1x1 = nn.Conv2d(out_channels, out_channels, kernel_size=1, padding=0)
        self.branch3x3 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.rcl_3x3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.branch_pool = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0)
        self.rcl_pool = nn.Conv2d(out_channels, out_channels, kernel_size=1, padding=0)
        self.bn = nn.BatchNorm2d(out_channels * 3)

    def forward(self, x):
        b1 = self.branch1x1(x)
        b3 = self.branch3x3(x)
        bp = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        bp = self.branch_pool(bp)
        for t in range(self.t_steps):
            b1 = self.rcl_1x1(b1) + b1
            b3 = self.rcl_3x3(b3) + b3
            bp = self.rcl_pool(bp) + bp
            if self.activation == "relu":
                b1 = F.relu(b1)
                b3 = F.relu(b3)
                bp = F.relu(bp)
            else:
                b1 = F.elu(b1)
                b3 = F.elu(b3)
                bp = F.elu(bp)
        out = torch.cat([b1, b3, bp], dim=1)
        out = self.bn(out)
        if x.shape[1] == out.shape[1]:
            out = out + x
        return out

In [7]:
class TransitionUnit(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_rate=0.5):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout2d(dropout_rate)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.pool(x)
        return x

class IRRCNN(nn.Module):
    def __init__(self, num_classes=2, activation="relu"):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.irr1 = InceptionRCLBlock(64, 128, t_steps=2, activation=activation)
        self.trans1 = TransitionUnit(128 * 3, 256)
        self.irr2 = InceptionRCLBlock(256, 256, t_steps=2, activation=activation)
        self.trans2 = TransitionUnit(256 * 3, 512)
        self.irr3 = InceptionRCLBlock(512, 512, t_steps=2, activation=activation)
        self.trans3 = TransitionUnit(512 * 3, 1024)
        self.irr4 = InceptionRCLBlock(1024, 1024, t_steps=2, activation=activation)
        self.global_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(1024 * 3, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.irr1(x)
        x = self.trans1(x)
        x = self.irr2(x)
        x = self.trans2(x)
        x = self.irr3(x)
        x = self.trans3(x)
        x = self.irr4(x)
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [8]:
# optimizer, scheduler, loss, amp, device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = IRRCNN(num_classes=2, activation="relu").to(device)
optimizer = SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
scheduler = StepLR(optimizer, step_size=50, gamma=0.1)
criterion = nn.CrossEntropyLoss()
scaler = torch.amp.GradScaler(enabled=USE_AMP)

In [9]:
# Training and evaluation loops (unchanged)
def train_one_epoch(model, loader, optimizer, criterion, scaler, device, epoch):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    loop = tqdm(loader, desc=f"Epoch {epoch} [train]", leave=True)
    for batch_idx, (images, labels) in enumerate(loop):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        with torch.amp.autocast(device_type='cuda', dtype=torch.float16, enabled=USE_AMP):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        loop.set_postfix(loss=loss.item(), acc=correct/total)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def evaluate(model, loader, criterion, device):
    model.eval()
    correct, total = 0, 0
    loop = tqdm(loader, desc="Validation", leave=True)
    with torch.no_grad():
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            loop.set_postfix(acc=correct/total)
    acc = correct / total
    return acc

In [10]:
# --- NEW: patient-level evaluation function ---
def compute_patient_level_accuracy(model, wrapped_subset, full_dataset, device):
    """
    Compute patient-level score as:
      For each patient p:
        Nncp = total images of patient p
        Nntp = number of correctly classified images for patient p
        Ps_p = Nntp / Nncp
      Global patient recognition rate Prt = (sum_p Ps_p) / Nnp

    Inputs:
      model: trained model (in eval mode)
      wrapped_subset: instance of WrappedDataset that wraps a Subset of full_dataset
      full_dataset: the original BreakHisDataset (to access samples paths)
      device: torch device
    Returns:
      prt (float): global patient recognition rate
      patient_stats (dict): per-patient dict with entries {patient_id: (Nncp, Nntp, Ps_p)}
    """
    model.eval()
    # Dictionary to accumulate counts: patient_id -> {'n':Nncp, 'correct':Nntp}
    patient_counts = {}

    # We'll iterate over the wrapped_subset in order (0..len-1).
    # To recover original sample index in full_dataset for i-th item:
    #   orig_idx = wrapped_subset.subset.indices[i]
    subset_obj = wrapped_subset.subset  # this is a torch.utils.data.Subset

    with torch.no_grad():
        for i in range(len(wrapped_subset)):
            img, label = wrapped_subset[i]  # transform already applied by WrappedDataset
            orig_idx = subset_obj.indices[i]  # index in full_dataset.samples
            img_path, _ = full_dataset.samples[orig_idx]  # original path and label
            # extract patient id: parent folder of MAGNIFICATION folder
            # path layout: .../<binary_class>/SOB/<class_folder>/<patient_folder>/<MAGNIFICATION>/<image>
            # so patient_folder = dirname(dirname(img_path))
            patient_folder_path = os.path.dirname(os.path.dirname(img_path))
            patient_id = os.path.basename(patient_folder_path)

            # forward
            img = img.unsqueeze(0).to(device)  # add batch dim
            outputs = model(img)
            pred = outputs.argmax(dim=1).item()

            # update patient counts
            if patient_id not in patient_counts:
                patient_counts[patient_id] = {'n': 0, 'correct': 0}
            patient_counts[patient_id]['n'] += 1
            if pred == int(label):
                patient_counts[patient_id]['correct'] += 1

    # compute Ps per patient and Prt
    patient_stats = {}
    sum_ps = 0.0
    for pid, vals in patient_counts.items():
        Nncp = vals['n']
        Nntp = vals['correct']
        Ps_p = Nntp / Nncp if Nncp > 0 else 0.0
        patient_stats[pid] = (Nncp, Nntp, Ps_p)
        sum_ps += Ps_p

    Nnp = len(patient_counts)
    Prt = sum_ps / Nnp if Nnp > 0 else 0.0
    return Prt, patient_stats

In [11]:
# --- Main training loop (same logic) ---
best_acc = 0.0
for epoch in range(1, NUM_EPOCHS+1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, scaler, device, epoch)
    val_acc = evaluate(model, valid_loader, criterion, device)
    scheduler.step()
    print(f"Epoch {epoch}: Validation Accuracy = {val_acc:.4f}")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_irrcnn.pt")

Validation: 100%|██████████| 13/13 [00:09<00:00,  1.35it/s, acc=0.894]


Epoch 1: Validation Accuracy = 0.8937


Validation: 100%|██████████| 13/13 [00:06<00:00,  1.88it/s, acc=0.811]


Epoch 2: Validation Accuracy = 0.8106


Validation: 100%|██████████| 13/13 [00:07<00:00,  1.85it/s, acc=0.841]


...

Epoch 148: Validation Accuracy = 0.9103


Validation: 100%|██████████| 13/13 [00:06<00:00,  1.86it/s, acc=0.91] 


Epoch 149: Validation Accuracy = 0.9103


Validation: 100%|██████████| 13/13 [00:06<00:00,  1.88it/s, acc=0.914]

Epoch 150: Validation Accuracy = 0.9136


In [12]:
# Load best and evaluate
model.load_state_dict(torch.load("best_irrcnn.pt", map_location=device))

# Patient-level evaluation on test set
prt, patient_stats = compute_patient_level_accuracy(model, test_dataset, full_dataset, device)
print(f"Final Patient-level Recognition Rate (Prt): {prt:.4f}")
# optionally: print per-patient summary (few examples)
# print("Sample of patient-level stats (patient_id: N_images, N_correct, Ps):")
# for i, (pid, stats) in enumerate(patient_stats.items()):
#     print(f"{pid}: {stats[0]}, {stats[1]}, {stats[2]:.4f}")
#     if i >= 9: break  # show only first 10 for brevity

Final Patient-level Recognition Rate (Prt): 0.9154
